# Test NhanhVN Product List API (v3.0/product/list)

API này chính xác hơn /product/inventory và có inventory data trong response

In [2]:
import requests
import json
from pprint import pprint
import time
# Credentials từ .env
APP_ID = 76531  # VITE_NHANH_APP_ID
BUSINESS_ID = 209189  # VITE_NHANH_BUSINESS_ID
ACCESS_TOKEN = "JZu1Qvqaw9guvDMEvWMStxuuHoTUndZ5S9WNUKlKLb2Y81llfCReYMRfRL7DjRUWXos5DHxrasK8u2ZRxTH8xNgsmNU8YlMeSnVQgHTHEoT53qQjzkZz7SUoSc0Gj8xrCo87fzOx1fsYsOmFpdnYij4rYZYjvWFHpAz4wznikdpNs1JABwijty3lqH86jBGjXGaYZVC1x1VFWSi0mb7wIpp3LF5d3YAh9vg26p7QKie"   # VITE_NHANH_ACCESS_TOKEN

BASE_URL = "https://pos.open.nhanh.vn/v3.0"

print(f"✅ APP_ID: {APP_ID}")
print(f"✅ BUSINESS_ID: {BUSINESS_ID}")
print(f"✅ ACCESS_TOKEN: {ACCESS_TOKEN[:20]}...")

✅ APP_ID: 76531
✅ BUSINESS_ID: 209189
✅ ACCESS_TOKEN: JZu1Qvqaw9guvDMEvWMS...


In [3]:

class NhanhVnService:
    def __init__(self, app_id, business_id, access_token):
        self.app_id = app_id
        self.business_id = business_id
        self.access_token = access_token
        self.base_url = "https://pos.open.nhanh.vn/v3.0"
        self.headers = {
            'Authorization': self.access_token,
            'Content-Type': 'application/json'
        }

    def _call_api(self, endpoint, payload):
        """Hàm chung để gọi API POST"""
        url = f"{self.base_url}{endpoint}?appId={self.app_id}&businessId={self.business_id}"
        try:
            response = requests.post(url, headers=self.headers, data=json.dumps(payload))
            response.raise_for_status() # Báo lỗi nếu HTTP status != 200
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"[Lỗi kết nối] {e}")
            return None

    def get_all_active_categories(self):
        """Lấy toàn bộ danh mục đang hoạt động (status=1)"""
        print("--- Đang lấy danh sách danh mục ---")
        endpoint = "/product/category"
        all_categories = []
        next_page = None

        while True:
            payload = {
                "filters": {
                    "status": 1  # Chỉ lấy danh mục Active
                },
                "paginator": {
                    "size": 50 # Tối ưu tốc độ
                }
            }
            
            # Nếu có trang tiếp theo, thêm vào payload
            if next_page:
                payload["paginator"]["next"] = next_page

            data = self._call_api(endpoint, payload)
            
            if not data or data.get('code') != 1:
                print(f"Lỗi lấy danh mục: {data.get('messages') if data else 'Unknown'}")
                break

            items = data.get('data', [])
            all_categories.extend(items)
            print(f"Đã tải {len(items)} danh mục...")

            # Kiểm tra phân trang
            paginator_resp = data.get('paginator', {})
            next_page = paginator_resp.get('next')
            
            # Nếu không còn trang sau hoặc next rỗng thì dừng
            if not next_page:
                break
            
        print(f"-> Tổng cộng: {len(all_categories)} danh mục.")
        return all_categories

    def get_inventory_by_category(self, category_id, category_name):
        """Lấy tồn kho theo ID danh mục"""
        print(f"--- Đang lấy tồn kho cho danh mục: {category_name} (ID: {category_id}) ---")
        endpoint = "/product/inventory"
        all_products = []
        next_page = None

        while True:
            payload = {
                "filters": {
                    "categoryId": category_id # Filter theo danh mục
                },
                "paginator": {
                    "size": 50
                }
            }

            if next_page:
                payload["paginator"]["next"] = next_page

            data = self._call_api(endpoint, payload)

            if not data or data.get('code') != 1:
                # Có thể danh mục này không có sản phẩm nào, hoặc lỗi
                break

            items = data.get('data', [])
            all_products.extend(items)

            paginator_resp = data.get('paginator', {})
            next_page = paginator_resp.get('next')

            if not next_page:
                break
        
        print(f"-> Tìm thấy {len(all_products)} sản phẩm trong danh mục '{category_name}'.")
        return all_products

# --- CHẠY THỬ ---
if __name__ == "__main__":
    # CẤU HÌNH THÔNG TIN CỦA BẠN
    APP_ID = 76531  # VITE_NHANH_APP_ID
    BUSINESS_ID = 209189  # VITE_NHANH_BUSINESS_ID
    ACCESS_TOKEN = "JZu1Qvqaw9guvDMEvWMStxuuHoTUndZ5S9WNUKlKLb2Y81llfCReYMRfRL7DjRUWXos5DHxrasK8u2ZRxTH8xNgsmNU8YlMeSnVQgHTHEoT53qQjzkZz7SUoSc0Gj8xrCo87fzOx1fsYsOmFpdnYij4rYZYjvWFHpAz4wznikdpNs1JABwijty3lqH86jBGjXGaYZVC1x1VFWSi0mb7wIpp3LF5d3YAh9vg26p7QKie"   # VITE_NHANH_ACCESS_TOKEN


    service = NhanhVnService(APP_ID, BUSINESS_ID, ACCESS_TOKEN)

    # 1. Lấy danh sách danh mục trước
    categories = service.get_all_active_categories()

    # 2. Duyệt qua từng danh mục để lấy tồn kho
    full_inventory_data = {} # Dictionary lưu kết quả: { "Tên danh mục": [Danh sách sản phẩm] }

    for cat in categories:
        cat_id = cat.get('id')
        cat_name = cat.get('name')
        
        # Gọi hàm lấy tồn kho
        products = service.get_inventory_by_category(cat_id, cat_name)
        
        if products:
            full_inventory_data[cat_name] = products
            
        # Nên sleep nhẹ 1 chút để tránh bị chặn rate limit nếu dữ liệu quá lớn
        # time.sleep(0.5) 

    # In thử kết quả của 1 danh mục đầu tiên để kiểm tra
    if full_inventory_data:
        first_cat = list(full_inventory_data.keys())[0]
        print(f"\nVí dụ dữ liệu tồn kho của danh mục '{first_cat}':")
        print(json.dumps(full_inventory_data[first_cat][:2], indent=2, ensure_ascii=False)) # In 2 sp đầu
    else:
        print("\nKhông tìm thấy dữ liệu tồn kho nào.")


--- Đang lấy danh sách danh mục ---
Đã tải 34 danh mục...
Đã tải 0 danh mục...
-> Tổng cộng: 34 danh mục.
--- Đang lấy tồn kho cho danh mục: Quần Gió (ID: 36) ---
-> Tìm thấy 3 sản phẩm trong danh mục 'Quần Gió'.
--- Đang lấy tồn kho cho danh mục: Ba lô (ID: 34) ---
-> Tìm thấy 2 sản phẩm trong danh mục 'Ba lô'.
--- Đang lấy tồn kho cho danh mục: Tất (ID: 33) ---
-> Tìm thấy 7 sản phẩm trong danh mục 'Tất'.
--- Đang lấy tồn kho cho danh mục: Quần Nỉ Dài (ID: 32) ---
-> Tìm thấy 12 sản phẩm trong danh mục 'Quần Nỉ Dài'.
--- Đang lấy tồn kho cho danh mục: Quần Lót (ID: 31) ---
-> Tìm thấy 6 sản phẩm trong danh mục 'Quần Lót'.
--- Đang lấy tồn kho cho danh mục: Quần Kaki (ID: 30) ---
-> Tìm thấy 9 sản phẩm trong danh mục 'Quần Kaki'.
--- Đang lấy tồn kho cho danh mục: Quần jogger (ID: 29) ---
-> Tìm thấy 2 sản phẩm trong danh mục 'Quần jogger'.
--- Đang lấy tồn kho cho danh mục: Mũ (ID: 28) ---
-> Tìm thấy 4 sản phẩm trong danh mục 'Mũ'.
--- Đang lấy tồn kho cho danh mục: Thắt lưng (ID: 2